In [ ]:
import polars as pl
from huggingface_hub import snapshot_download
from datasets import load_dataset
import os
import sys
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
sys.path.append("..")
%load_ext autoreload
%autoreload 2


In [ ]:
import os

from synthetic_data.tasks.writing import _get_gutenberg_subset, find_valid_chunks

gutenberg_pq: pl.DataFrame = _get_gutenberg_subset(2)

In [ ]:
gutenberg_pq.columns

In [ ]:
import tiktoken
from synthetic_data.gutenberg_parser import super_cleaner
from IPython.display import Markdown

from synthetic_data.tasks.writing import _get_chunks_gutenberg

tiktoken_encoder = tiktoken.get_encoding("o200k_base")

for row in gutenberg_pq.sample(100).iter_rows(named=True):
    chunks = _get_chunks_gutenberg(row, tiktoken_encoder)
    display(Markdown(f"**{row['title']}**"))
    for chunk in chunks:
        n_tokens = sum([len(tiktoken_encoder.encode(par)) for par in chunk])
        print("-"*100)
        print(f"n_tokens: {n_tokens}")
        for par in chunk:
            tokens = tiktoken_encoder.encode(par)
            display(Markdown(par))
        break


In [ ]:
import re

from synthetic_data.utils import ldictl
from synthetic_data.tasks.writing import GutenbergBacktranslation
first_sample = gutenberg_pq.sample(n=1)
task = GutenbergBacktranslation()

input = first_sample.to_dicts()
formatted_out = task.format_input_conversation(ldictl(input))

In [ ]:
formatted_out[0]